In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [2]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser") # Contenido de la pagina
parquet_files = [] # Creo una lista para alojar todos los links que sean de los archivos parquet

In [3]:
# Hago una filtración de los archivos que pertenecen solamente a los años que nos importan
selected_years = ['2021', '2022', '2023']


for link in soup.find_all('a',href=True): # Busco todos los elementos que sean links
  if link['href'].endswith('.parquet'): # Si el elemento termina en .parquet, añado el elemento a la lista parquet_files
    if any(year in link['href'] for year in selected_years):
      parquet_files.append(link['href'])
print(f'{len(parquet_files)} links parquet se cargaron')

48 links parquet se cargaron


In [4]:
numero_de_elementos_parquet= len(parquet_files)
numero_de_elementos_parquet

48

In [6]:

yellow_taxis = []
green_taxis = []
fhv_taxis = []
fhvhv_taxis = []

for link in parquet_files:
  if 'yellow_tripdata' in link: # Todos los links que contengan 'yellow_tripdata' en su texto
    yellow_taxis.append(link)
  elif 'green_tripdata' in link: # Todos los links que contengan 'green_tripdata' en su texto
    green_taxis.append(link)
  elif 'fhv_tripdata' in link: # Todos los links que contengan 'fhv_tripdata' en su texto
    fhv_taxis.append(link)
  elif 'fhvhv_tripdata' in link: # Todos los links que contengan 'fhvhv_tripdata' en su texto
    fhvhv_taxis.append(link)

yellow_taxis = sorted(yellow_taxis, reverse=True)
green_taxis = sorted(green_taxis, reverse=True)
fhv_taxis = sorted(fhv_taxis, reverse=True)
fhvhv_taxis = sorted(fhvhv_taxis, reverse=True)
     

In [7]:
def download_parquet_links(links_list, taxi_color):
    # Create a variable to store the links
    parquet_links = links_list

    # Print a message confirming download
    print(f'Se descargaron los enlaces de parquet para {taxi_color}')

    return parquet_links


In [8]:
# Define variables to store the links
yellow_links = download_parquet_links(yellow_taxis, 'yellow')
green_links = download_parquet_links(green_taxis, 'green')

# Read the parquet files
taxi1 = pd.read_parquet(yellow_links[-1])
taxi1['Color'] = 'yellow'

taxi2 = pd.read_parquet(green_links[-1])
taxi2['Color'] = 'green'


Se descargaron los enlaces de parquet para yellow
Se descargaron los enlaces de parquet para green


In [9]:
taxi1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1369769 entries, 0 to 1369768
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1369769 non-null  int64         
 1   tpep_pickup_datetime   1369769 non-null  datetime64[us]
 2   tpep_dropoff_datetime  1369769 non-null  datetime64[us]
 3   passenger_count        1271417 non-null  float64       
 4   trip_distance          1369769 non-null  float64       
 5   RatecodeID             1271417 non-null  float64       
 6   store_and_fwd_flag     1271417 non-null  object        
 7   PULocationID           1369769 non-null  int64         
 8   DOLocationID           1369769 non-null  int64         
 9   payment_type           1369769 non-null  int64         
 10  fare_amount            1369769 non-null  float64       
 11  extra                  1369769 non-null  float64       
 12  mta_tax                13697

In [12]:
taxi2.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Color
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00,green
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75,green
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00,green
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00,green
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,...,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00,green


In [6]:
def download_parquet_links(links_list, taxi_color):
  with open(f'{taxi_color}.json', 'w') as json_file:
    json.dump(links_list, json_file)
  print(f'Se descargo un archivo llamado "{taxi_color}_parquet_links.json"')

In [7]:
download_parquet_links(yellow_taxis, 'yellow')
download_parquet_links(green_taxis, 'green')
download_parquet_links(fhv_taxis, 'fhv')
download_parquet_links(fhvhv_taxis, 'fhvhv')

Se descargo un archivo llamado "yellow_parquet_links.json"
Se descargo un archivo llamado "green_parquet_links.json"
Se descargo un archivo llamado "fhv_parquet_links.json"
Se descargo un archivo llamado "fhvhv_parquet_links.json"


In [8]:
taxi1=pd.read_parquet(yellow_taxis[-1])
taxi1['Color'] = 'yellow'

In [9]:
taxi2=pd.read_parquet(green_taxis[-1])
taxi1['Color'] = 'Green'

In [10]:
taxi3=pd.read_parquet(fhv_taxis[-1])
taxi1['Color'] = 'fhv'

In [11]:
taxi4=pd.read_parquet(fhvhv_taxis[-1])
taxi1['Color'] = 'fhvhv'

In [12]:
dftaxis= pd.concat([taxi1, taxi2, taxi3, taxi4], ignore_index=True)

In [13]:
dftaxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14508867 entries, 0 to 14508866
Data columns (total 49 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   VendorID                float64       
 1   tpep_pickup_datetime    datetime64[us]
 2   tpep_dropoff_datetime   datetime64[us]
 3   passenger_count         float64       
 4   trip_distance           float64       
 5   RatecodeID              float64       
 6   store_and_fwd_flag      object        
 7   PULocationID            float64       
 8   DOLocationID            float64       
 9   payment_type            float64       
 10  fare_amount             float64       
 11  extra                   float64       
 12  mta_tax                 float64       
 13  tip_amount              float64       
 14  tolls_amount            float64       
 15  improvement_surcharge   float64       
 16  total_amount            float64       
 17  congestion_surcharge    float64       
 18  